# Python Talk: Classes & Metaclasses

A class, to begin with, is a namespace. But note there is no access control (`public`/`private`/`protected` etc as in more “pure” object-oriented languages).

In [ ]:
class FirstClass :

    field = "something"

    def func(a, b) :
        print("a =", a, "and b =", b)
    #end func

#end FirstClass

(Notice the common Python convention that class names use camelcase (of the bactrian, not the dromedary, variety), while other names use lower-case plus underscores.)

In [ ]:
FirstClass.field

Unlike more conventional OO languages, a *method* is just a function that happens to find itself inside a class. It can still be invoked like any other function:

In [ ]:
FirstClass.func(2, 3)

Standard OO operation of *Instantiating* a class — creating an *instance* of that class — in Python looks like calling the class as a function (no need for `new` keyword as in Java and PHP):

In [ ]:
f = FirstClass()

In [ ]:
type(f)

In [ ]:
type(f) is FirstClass

In [ ]:
f.field

Now functions start to behave more like *methods* as you would expect in OO languages (this magic happens because functions are *descriptors*, which will be explained later):

In [ ]:
f.func(2, 3)

In [ ]:
f.func(3)

Method call is effectively just syntactic sugar for following function call:

In [ ]:
FirstClass.func(f, 3)

Or even:

In [ ]:
type(f).func(f, 3)

So, a function becomes an *instance method* just from the way it is called—via a class instance. In this situation, the class instance is passed as the first argument to the function.

Contrast other languages like C and Java, where there is a special `this` keyword for referring to the current class instance—Python has no such special mechanism. This to me is the sign of a language that acquired functions before it acquired classes...

The first argument to the definition of a function intended for use as a method is commonly named something like `self`, but there is no requirement in the language for this.

Constructor method is called `__init__` -- perhaps also mention `__del__` if cleanup needed. Warning about unpredictable environment when latter is invoked at script termination time.

In [ ]:
class SecondClass :
    
    c = "something"

    def __init__(self, i) :
        self.i = i
    #end __init__

    def func1(self, b) :
        print("self =", self, "and b =", b)
    #end func

#end SecondClass

In [ ]:
s = SecondClass(9)

In [ ]:
s.c

In [ ]:
s.i

In [ ]:
s.func1("x")

An instance method is itself a function, and like any function, is a first-class object:

In [ ]:
fn = s.func1
fn("y")

In [ ]:
s2 = SecondClass(10)

In [ ]:
s2.i

In [ ]:
s2.c = "something else"
s.c

For `s2`, there are now two variables called “c” -- the instance variable in `s2` itself, and the class variable in `SecondClass`. The former “hides” the latter.

In [ ]:
SecondClass.c = "something else"
s.c

Subclasses and superclasses; multiple inheritance; `super()` function

I haven’t made much use of multiple inheritance myself. Python does full-fat multiple inheritance à la C++, rather than the “lite” form in Java (interfaces).

Simple example of multiple inheritance:

In [ ]:
class ThirdClass(FirstClass, SecondClass) :

    def __init__(self, i, j) :
        super().__init__(i) # explicit call to superclass constructor
        self.j = j
    #end __init__

    def func2(self) :
        print("hi there!")
    #end func2

#end ThirdClass

In [ ]:
s = ThirdClass(8, 9)

In [ ]:
isinstance(s, FirstClass)

In [ ]:
isinstance(s, SecondClass)

In [ ]:
s.func1(3)

In [ ]:
s.func2()

In [ ]:
ThirdClass.__bases__

In [ ]:
ThirdClass.__mro__

Then there is also `__new__`, for taking greater control of class instantiation. For example, here is how you could define a “singleton” class, where every attempt at instantiation returns the same instance:

In [ ]:
class MySingleton :

    single_instance = None

    def __new__(cself) :
        # implicitly a classmethod
        print("MySingleton.__new__")
        if cself.single_instance == None :
            cself.single_instance = super().__new__(cself)
            self = cself.single_instance
            print("MySingleton instance created")
        else :
            print("Reusing same instance")
        #end if
        return cself.single_instance
    #end __new__

    # note no __init__ method, all instance initialization done in __new__
    # because __init__ would be called every time caller tried to create a
    # new instance

#end MySingleton

class NonSingleton :
    pass
#end NonSingleton

In [ ]:
a = MySingleton()
b = MySingleton()
print("MySingleton instances are equal? ", a is b)

a = NonSingleton()
b = NonSingleton()
print("NonSingleton instances are equal? ", a is b)


dynamic addition of attributes to class instances -- versus classes themselves?

In [ ]:
s2.d = "new field"

In [ ]:
s.d

In [ ]:
s2.d

introspection — builtin functions `dir`, `hasattr`, `getattr`, `setattr`, `delattr`

In [ ]:
dir(s)

In [ ]:
dir(s2)

Peculiarity of scoping when trying to refer within inner class to definition in outer class — because class definition hasn’t finished yet.

In [ ]:
del a, b # avoid interference from above

In [ ]:
class Outer :

    a = 1
    
    class Inner :
        
        b = a # can’t do that here
        
    #end Inner
    
#end Outer

In [ ]:
class Outer :

    a = 1
    
    class Inner :
        
        b = Outer.a # still won’t work
        
    #end Inner
    
#end Outer

In [ ]:
class Outer :

    a = 1
    
    class Inner :

        # b = a # can’t do that here
        pass
        
    #end Inner

    Inner.b = a # this works
    
#end Outer


What this really means is that classes-within-classes don’t have the special behaviours that they do in C++ and Java: in both those languages they get access to non-`public` members of the outer class (which don’t exist in Python), and in Java they further get tied (if non-`static`) to a specific instance of the outer class (effectively each instance of the outer class gets its own unique inner class), which again is meaningless in Python. In Python this nesting is nothing more than an indication of some kind of grouping relationship.

But note that, in Python, there are no “declarations” as such: all statements--which includes class and function declarations--are executable. Re-executing them creates new classes/functions, and the identifiers you use to name them are just variables that are assigned those classes/functions as values. This makes it useful, for example, to put a class definition inside a function definition, to be returned as the function result--the function becomes a class factory!

So, to get the equivalent of Java-style non-`static` inner classes, you could do something like

In [ ]:
class Outer :

    def __init__(self, v) :
        self.v = v
        
        class Inner :
            parent = self
            
            def __init__(self, x) :
                self.x = x
            #end __init__
            
            def sum(self) :
                return self.parent.v + self.x
            #end sum
            
        #end Inner
        
        self.Inner = Inner
    #end __init__

#end Outer

In [ ]:
o1 = Outer(10)
o2 = Outer(20)
i1 = o1.Inner(3)
i2 = o2.Inner(3)
i2a = o2.Inner(4)
print(i1.sum())
print(i2.sum())
print(i2a.sum())
o2.v = 30
print(i1.sum())
print(i2.sum())
print(i2a.sum())


Notice in the above how `i2` and `i2a` come from the same parent `Outer` instance, giving them the same value of the class variable `v`.

Previously discussed [special dunder method names](https://docs.python.org/3/reference/datamodel.html#special-method-names) for overloading of operators and other standard language constructs — perhaps skip details for today

Mention [descriptors](https://docs.python.org/3/reference/datamodel.html#implementing-descriptors) — can be a bit abstract, but important for understanding how methods and properties work.

Every function is a (non-data) descriptor. Properties are data descriptors: difference is that former, as methods, can be overridden by creating a class instance attribute of same name, latter can’t.

In [ ]:
dir(FirstClass.func) # has “__get__” but no “__set__”

*Decorators* -- can be applied to functions or classes:

    @«expr»
    def func() :
        ....

is equivalent to

    def func() :
        ....
    #end func
    func = «expr»(func)

and similarly

    @«expr»
    class Class() :
        ....

is equivalent to

    class Class() :
        ....
    #end Class
    Class = «expr»(Class)

So *«expr»* is any Python expression that returns a *callable* of a single argument; this is applied to the class or function being decorated, and is expected to return the suitably processed class/function (or an entirely new class/function), which is assigned back to the same class/function variable.

Example use of custom class and method decorators: my [DBussy](https://github.com/ldo/dbussy) binding for libdbus, particularly the `bigben_server_ravelled`, `bigben_listener_ravelled` and `bigben_listener_ravelled_alt` from the [example scripts](https://github.com/ldo/dbussy_examples). Show how you can introspect the server and automatically get an XML description of all its interfaces, without having to write any XML.

Instance methods versus class methods versus static methods — specified via standard decorators
  * instance methods are the default for a function inside a class
  * `@staticmethod` turns the following function into a static method
  * `@classmethod` turns the following function into a class method
  
`staticmethod` and `classmethod` are just built-in Python functions, designed to be used as decorators, but you can call them as normal functions.

*Static* methods are just functions that happen to find themselves inside a class, and calls to them via instances behave just as normal function calls. In other languages the point to having them is they have access to non-public class members. Since there are no such things in Python, its static methods are just a grouping mechanism — this function has something to do with this class, so make that clear by putting it here.

*Class* methods are something unique to Python — when called via a class instance, they get the *class* itself as the first argument, rather than the instance. Why bother, when the function can directly refer to the class itself by name? In fact this is handy when subclassing — in this situation, the inherited class method gets the subclass as its first argument, not the superclass where the method was actually defined.

Example? Factory methods in my [Qahirah](https://github.com/ldo/qahirah) binding. *E.g.* various ways of creating a `Vector` object: direct specification of *x* and *y* components; specifying them as a tuple; creating a unit `Vector` in a specified direction; or giving the polar coordinates. `Vector` is subclassed as the `Point` type in my [`python_pixman`](https://github.com/ldo/python_pixman) library; each class defines its own methods for converting to/from the underlying library data type, but they can share common Python behaviour.

In [ ]:
from qahirah import \
    deg, \
    Vector
from pixman import \
    Point

In [ ]:
v = Vector.unit(45 * deg)
p = Point.unit(45 * deg)
isinstance(p, Vector)

In [ ]:
v == p

In [ ]:
v + p

In [ ]:
p.to_pixman_fixed()

In [ ]:
v.to_pixman_fixed()

Properties — the other common use of descriptors. Again, examples from Qahirah. Read-only properties are a minor convenience, being able to type *obj*`.`*prop* instead of *obj*`.`*prop*`()`. Read-write properties are more of a convenience.

*E.g.* qahirah.Context.dash? Time to mention usefulness of specifying `__slots__` to preempt typos (`dashes` versus `dash`)?

In [ ]:
import qahirah

ctx = qahirah.Context.create_for_dummy()
ctx.dash

In [ ]:
ctx.dash = ([1, 1], 0.5)
ctx.dash

In [ ]:
ctx.dashes = ([1, 1], 0.5)

[Metaclasses](https://docs.python.org/3/reference/datamodel.html#customizing-class-creation):
  * Every value in Python is an object
  * Every object is an instance of a class
  * Classes are also values, hence objects, that can be generated and manipulated at runtime
  * Therefore classes must also be instances of classes
  * The class that a class is an instance of is the *metaclass* of the class
  * Default metaclass is called `type` (one of 2 meanings of this built-in function), but you can specify your own

Hard to think of an example use of metaclasses that isn’t more easily done with class decorators, but how about this contrived example: a kind of “enumeration” class with a fixed number of instances that are specified at class-definition time. The metaclass provides the mechanism for defining such classes:

In [ ]:
class FixedInstances(type) :
    "Use this as a metaclass on a class definition with an" \
    " “__instances__” attribute that names instances to be" \
    " constructed."

    def __new__(cself, name, bases, namespace) :
        result = type.__new__(cself, name, bases, dict(namespace))
        if not hasattr(result, "__real_init__") :
            # save the __init__ method under another name to block direct
            # instantiation
            result.__real_init__ = result.__init__
            def _dummyinit(self, *args, **kwargs) :
                "no new instances can be created."
                raise NotImplementedError("no new instances can be created")
            #end _dummyinit
            _dummyinit.__name__ = "__init__"
            result.__init__ = _dummyinit
        #end if
        for instance_name in namespace["__instances__"] :
            instance = result.__new__(result)
            instance.__real_init__(instance_name)
            setattr(result, instance_name, instance)
        #end for
        return result
    #end __new__

#end FixedInstances

Here is an example use of the metaclass. The class is expected to have an `__instances__` attribute which names the class instances to be created:

In [ ]:
class FixedInstancesExample(metaclass = FixedInstances) :

    def __init__(self, name) :
        self.text = "this is instance “%s” of the class" % name
    #end __init__

    def __repr__(self) :
        return self.text
    #end __repr__

    __instances__ = ("one", "two", "three")

#end FixedInstancesExample

In [ ]:
FixedInstancesExample.one

In [ ]:
isinstance(FixedInstancesExample.one, FixedInstancesExample)

In [ ]:
FixedInstancesExample()

In [ ]:
help(FixedInstancesExample.__init__)

I guess the clever thing about metaclasses is that they automatically come into action when you subclass a class that specified that metaclass, without having to be specified again — unlike a class decorator, which needs to be specified every place you want it to act. The following subclass continues the example, and adds its own instances:

In [ ]:
class FixedInstancesSubclass(FixedInstancesExample) :
    __instances__ = ("four", "five", "six")
#end FixedInstancesSubclass

Naturally it inherits the instances of the superclass:

In [ ]:
FixedInstancesSubclass.one

In [ ]:
type(FixedInstancesSubclass.one)

In [ ]:
FixedInstancesSubclass.four

In [ ]:
type(FixedInstancesSubclass.four)

In [ ]:
FixedInstancesSubclass()